In [1]:
import sys  
sys.path.insert(0, 'D:/TU_KL/thesis/cnn_compression')

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from models.DCT_alexnets_cifar10 import DCT_AlexNet
from scipy.fftpack import dct
import math
## cifar10 download problem solve
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
net = DCT_AlexNet(num_classes=10).to(device)
print(net)

DCT_AlexNet(
  (features): Sequential(
    (0): Conv2dDCT(in_channels=3, out_channels=64, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2dDCT(in_channels=64, out_channels=192, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2dDCT(in_channels=192, out_channels=384, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU(inplace=True)
    (8): Conv2dDCT(in_channels=384, out_channels=256, kernel_size=(3,), stride=(1,), padding=(1,))
    (9): ReLU(inplace=True)
    (10): Conv2dDCT(in_channels=256, out_channels=256, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    

In [5]:
def train(dataloader,model,criterion,optimizer):
#     torch.autograd.detect_anomaly() 
    train_loss = 0.0
    for X, y in dataloader:
        inputs, labels = X.to(device), y.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()*inputs.size(0)
    train_loss = train_loss/len(dataloader)
    
    print(f'Training Loss: {train_loss:.8f}')

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

print("DCT_AlexNet.......old dct_conv + linear_dct")
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, net, criterion, optimizer)
    test(testloader, net, criterion)


DCT_AlexNet.......old dct_conv + linear_dct
Epoch 1
-------------------------------


d:\TU_KL\thesis\cnn_comp\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training Loss: 73.48244739
Test Error: 
 Accuracy: 22.1%, Avg loss: 2.091920 

Epoch 2
-------------------------------
Training Loss: 63.19896191
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.832240 

Epoch 3
-------------------------------
Training Loss: 59.94092828
Test Error: 
 Accuracy: 35.3%, Avg loss: 1.802803 

Epoch 4
-------------------------------
Training Loss: 57.61938524
Test Error: 
 Accuracy: 34.3%, Avg loss: 1.774468 

Epoch 5
-------------------------------
Training Loss: 56.12502264
Test Error: 
 Accuracy: 35.3%, Avg loss: 1.784354 



In [8]:
## no. of paramaters
print('no of trainable params:', sum(p.numel() for p in net.parameters() if p.requires_grad))

no of trainable params: 59673


In [9]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(net)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|    features.0.fcc   |     64     |
|    features.0.fcl   |     3      |
|   features.0.bias   |     64     |
|    features.3.fcc   |    192     |
|    features.3.fcl   |     3      |
|   features.3.bias   |    192     |
|    features.6.fcc   |    384     |
|    features.6.fcl   |     3      |
|   features.6.bias   |    384     |
|    features.8.fcc   |    256     |
|    features.8.fcl   |     3      |
|   features.8.bias   |    256     |
|   features.10.fcc   |    256     |
|   features.10.fcl   |     3      |
|   features.10.bias  |    256     |
|   classifier.1.fc   |    4096    |
|  classifier.1.bias  |    4096    |
|   classifier.4.fc   |    4096    |
|  classifier.4.bias  |    4096    |
| classifier.6.weight |   40960    |
|  classifier.6.bias  |     10     |
+---------------------+------------+
Total Trainable Params: 59673


59673